<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/final/multi_cls_mamba2_780m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [4]:
!pip install causal-conv1d>=1.1.0

In [5]:
!pip install mamba-ssm

In [6]:
!pip install datasets evaluate accelerate

In [7]:
!pip install huggingface_hub

In [8]:
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libumf.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is

In [9]:
import os
import random
import json
import torch
import torch.nn as nn
from collections import namedtuple
from dataclasses import dataclass, field, asdict
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
from mamba_ssm.utils.hf import load_config_hf, load_state_dict_hf

import evaluate
import numpy as np
from datasets import load_dataset
from transformers import Trainer
from transformers import AutoTokenizer, TrainingArguments

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
import wandb
wandb.login(key="ee23d2708eaedb89d7020a87e3ac4dea2b742261")
wandb.init(project="Multi output classification", entity="ai_expert", name="mamba2-780m")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ebinna-lee (ebinna-lee-x). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ebinna-lee (ai_expert). Use `wandb login --relogin` to force relogin


In [13]:
from dataclasses import dataclass, asdict
import json

@dataclass
class MambaConfig:
    d_model: int = 1536
    d_intermediate: int = 0
    n_layer: int = 48
    vocab_size: int = 50277
    ssm_cfg: dict = None
    attn_layer_idx: list = None
    attn_cfg: dict = None
    rms_norm: bool = True
    residual_in_fp32: bool = True
    fused_add_norm: bool = True
    pad_vocab_size_multiple: int = 16
    tie_embeddings: bool = True

    def __post_init__(self):
        # 기본값으로 설정된 None 타입을 빈 딕셔너리와 빈 리스트로 초기화
        if self.ssm_cfg is None:
            self.ssm_cfg = {"layer": "Mamba2"}
        if self.attn_layer_idx is None:
            self.attn_layer_idx = []
        if self.attn_cfg is None:
            self.attn_cfg = {}

    def to_json_string(self):
        return json.dumps(asdict(self))

    def to_dict(self):
        return asdict(self)

In [14]:
classes = ['Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS']
class2id = {'Algorithms' :0, 'Backend' : 1, 'Data Science' : 2, 'Databases' : 3, 'Dev Tools' : 4, 'Frontend' : 5, 'Mobile' :6, 'Systems' : 7, 'iOS/macOS' : 8}
id2class = {0 : 'Algorithms', 1: 'Backend', 2 : 'Data Science', 3 : 'Databases', 4 : 'Dev Tools', 5 : 'Frontend', 6 : 'Mobile', 7 : 'Systems', 8 :'iOS/macOS'}

In [15]:
class MambaClassificationHead(nn.Module):
    def __init__(self, d_model, num_classes, **kwargs):
        super(MambaClassificationHead, self).__init__()
        self.classification_head = nn.Linear(d_model, num_classes, **kwargs)

    def forward(self, hidden_states):
        return self.classification_head(hidden_states)

In [16]:
class MambaTextClassification(MambaLMHeadModel):
    def __init__(
        self,
        config: MambaConfig,
        initializer_cfg=None,
        device=None,
        dtype=None,
    ) -> None:
        super().__init__(config, initializer_cfg, device, dtype)

        self.classification_head = MambaClassificationHead(d_model=config.d_model, num_classes=len(classes))
        del self.lm_head
        self.multi_label = True
        self.id2label = id2class
        self.class2id = class2id

    @classmethod
    def addMambaClassificationHead(cls, num_classes, id2label, class2id, multi_label):
        cls.classification_head = MambaClassificationHead
        del self.lm_head

    def forward(self, input_ids, attention_mask=None, labels=None):
        hidden_states = self.backbone(input_ids)
        mean_hidden_states = hidden_states.mean(dim=1)

        logits = self.classification_head(mean_hidden_states)
        if labels is None:
          ClassificationOutput = namedtuple("ClassificationOutput", ["logits"])
          return ClassificationOutput(logits=logits)
        else:
          ClassificationOutput = namedtuple("ClassificationOutput", ["loss", "logits"])
          if self.multi_label:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels)
          else:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
          return ClassificationOutput(loss=loss, logits=logits)

    def predict(self, text, tokenizer):
        input_ids = torch.tensor(tokenizer(text)['input_ids'], device='cuda')[None]
        with torch.no_grad():
          logits = self.forward(input_ids).logits[0]

        if self.multi_label:
          probabilities = torch.sigmoid(logits).cpu().numpy()
          predictions = (probabilities > 0.5).astype(int)
          return [self.id2label[i] for i, value in enumerate(predictions) if value == 1]
        else:
          label = np.argmax(logits.cpu().numpy())
          return self.id2label[label]

    @classmethod
    def from_pretrained(cls, pretrained_model_name, device=None, dtype=None, **kwargs):
        config_data = load_config_hf(pretrained_model_name)
        config = MambaConfig(**config_data)

        model = cls(config, device=device, dtype=dtype, **kwargs)

        model_state_dict = load_state_dict_hf(pretrained_model_name, device=device, dtype=dtype)
        model.load_state_dict(model_state_dict, strict=False)

        print("Newly initialized embedding:", set(model.state_dict().keys()) - set(model_state_dict.keys()))
        return model

In [17]:
model = MambaTextClassification.from_pretrained("state-spaces/mamba2-780m")
model.to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Newly initialized embedding: {'classification_head.classification_head.weight', 'classification_head.classification_head.bias'}


MambaTextClassification(
  (backbone): MixerModel(
    (embedding): Embedding(50288, 1536)
    (layers): ModuleList(
      (0-47): 48 x Block(
        (norm): RMSNorm()
        (mixer): Mamba2(
          (in_proj): Linear(in_features=1536, out_features=6448, bias=False)
          (conv1d): Conv1d(3328, 3328, kernel_size=(4,), stride=(1,), padding=(3,), groups=3328)
          (act): SiLU()
          (norm): RMSNorm()
          (out_proj): Linear(in_features=3072, out_features=1536, bias=False)
        )
      )
    )
    (norm_f): RMSNorm()
  )
  (classification_head): MambaClassificationHead(
    (classification_head): Linear(in_features=1536, out_features=9, bias=True)
  )
)

In [18]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
tokenizer.pad_token_id = tokenizer.eos_token_id

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
from datasets import load_dataset

dataset_train = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/rev_tag_training_samples.csv',
    split='train'
)
dataset_valid = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/rev_tag_validation_samples.csv',
    split='train'
)
dataset_test = load_dataset(
    'csv',
    data_files='/content/drive/My Drive/AiExpertCource/project/dataset/rev_tag_test_samples.csv',
    split='train'
)

In [20]:
import ast

def preprocess_function(example):
   all_labels =  ast.literal_eval(example['Tags_new'])
   labels = [0. for i in range(len(classes))]
   for label in all_labels:
       label_id = class2id[label]
       labels[label_id] = 1.

   samples = tokenizer(example['Title'] + ' ' + example['Body'], truncation=True, max_length=512)
   samples.pop('attention_mask')
   samples['labels'] = labels
   return samples

In [21]:
tokenized_train_dataset = dataset_train.map(preprocess_function)
tokenized_valid_dataset = dataset_valid.map(preprocess_function)
tokenized_test_dataset = dataset_test.map(preprocess_function)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [22]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["Title", "Body", "Tags_filtered", "Tags_list", "Tags_new", "Algorithms", "Backend", "Data Science", "Databases", "Dev Tools", "Frontend", "Mobile", "Systems", "iOS/macOS"])
tokenized_valid_dataset = tokenized_valid_dataset.remove_columns(["Title", "Body", "Tags_filtered", "Tags_list", "Tags_new", "Algorithms", "Backend", "Data Science", "Databases", "Dev Tools", "Frontend", "Mobile", "Systems", "iOS/macOS"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["Title", "Body", "Tags_filtered", "Tags_list", "Tags_new", "Algorithms", "Backend", "Data Science", "Databases", "Dev Tools", "Frontend", "Mobile", "Systems", "iOS/macOS"])

In [23]:
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
import numpy as np

def sigmoid(x):
   return 1/(1 + np.exp(-x))


def compute_metrics(eval_pred):

  predictions, labels = eval_pred
  predictions = sigmoid(predictions)
  predictions = (predictions > 0.5).astype(int)
  accuracy = accuracy_score(labels, predictions)
  precision, recall, f1_score_result, _ = precision_recall_fscore_support(labels, predictions, average='micro')
  precision_macro, recall_macro, f1_score_result_macro, _ = precision_recall_fscore_support(labels, predictions, average='macro')
  precision_weighted, recall_weighted, f1_score_result_weighted, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

  flat_predictions = predictions.reshape(-1)
  flat_labels = labels.reshape(-1)
  flat_accuracy = accuracy_score(flat_labels, flat_predictions)

  return {
        'flat_accuracy' : flat_accuracy,
        'accuracy' : accuracy,
        'micro_precision': precision,
        'micro_recall': recall,
        'micro_f1': f1_score_result,
        'macro_precision': precision_macro,
        'macro_recall': recall_macro,
        'macro_f1': f1_score_result_macro,
        'weighted_precision': precision_weighted,
        'weighted_recall': recall_weighted,
        'weighted_f1': f1_score_result_weighted,
        }

In [24]:
class MambaTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        input_ids = inputs.pop("input_ids")
        labels = inputs.pop('labels')
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        return (loss, outputs) if return_outputs else loss

    def save_model(self, output_dir = None, _internal_call = False):
        if output_dir is None:
            output_dir = self.args.output_dir

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        torch.save(self.model.state_dict(), f"{output_dir}/pytorch_model.bin")

        self.tokenizer.save_pretrained(output_dir)

        with open(f'{output_dir}/config.json', 'w') as f:
            json.dump(self.model.config.to_dict(), f)

In [25]:
output_dir = "multi_cls_mamba2-780m"

In [26]:
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
trainer = MambaTrainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics
    )

In [28]:
text =  """Can't get read to work properly in this script
  I have the following script:

echo "User already exists, delete user? [Y/n] "
read -r -n 1 yn
echo "$yn was pressed"
When I run the script:

Pressing y or Y (without [Enter]) echoes the character to the screen.
Pressing [Enter] moves the cursor to a new line, but the following echo command doesn't execute
Pressing [Enter] a second time causes the echo command to execute
I expect it to not require me to press [Enter] twice.

I have tried different variations including putting the prompt in the read command, but that was worse since the prompt never even showed up at that point.

What am I doing wrong?
});
"""

In [29]:
response = model.predict(text, tokenizer)
print(response)

['Backend', 'Databases']


In [30]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Flat Accuracy,Accuracy,Micro Precision,Micro Recall,Micro F1,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1
1,0.123400,0.103900,0.962889,0.730923,0.863457,0.861160,0.862307,0.764258,0.876226,0.793108,0.879077,0.861160,0.866322
2,0.056200,0.115521,0.962991,0.721385,0.838874,0.898277,0.867560,0.754064,0.898205,0.807060,0.843853,0.898277,0.869280
3,0.010300,0.152158,0.962444,0.715231,0.839471,0.892323,0.865091,0.754036,0.898917,0.806257,0.847334,0.892323,0.867539
4,0.002100,0.162022,0.964222,0.730615,0.848241,0.894984,0.870986,0.752042,0.897540,0.800480,0.856662,0.894984,0.873838
5,0.000400,0.165913,0.964598,0.731692,0.849730,0.896124,0.872310,0.761604,0.897995,0.809692,0.856967,0.896124,0.874690


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


TrainOutput(global_step=12500, training_loss=0.04505177847146988, metrics={'train_runtime': 15360.9066, 'train_samples_per_second': 6.51, 'train_steps_per_second': 0.814, 'total_flos': 0.0, 'train_loss': 0.04505177847146988, 'epoch': 5.0})

In [31]:
trainer.evaluate()

{'eval_loss': 0.1659129112958908,
 'eval_flat_accuracy': 0.9645982905982906,
 'eval_accuracy': 0.7316923076923076,
 'eval_micro_precision': 0.8497297297297297,
 'eval_micro_recall': 0.8961236382062325,
 'eval_micro_f1': 0.8723102534064986,
 'eval_macro_precision': 0.7616043820702195,
 'eval_macro_recall': 0.8979954936561685,
 'eval_macro_f1': 0.809692324727526,
 'eval_weighted_precision': 0.8569669674689603,
 'eval_weighted_recall': 0.8961236382062325,
 'eval_weighted_f1': 0.8746904791992751,
 'eval_runtime': 279.3743,
 'eval_samples_per_second': 23.266,
 'eval_steps_per_second': 2.91,
 'epoch': 5.0}

In [32]:
trainer_for_test = MambaTrainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics
    )

In [33]:
trainer_for_test.evaluate()

{'eval_loss': 0.17435696721076965,
 'eval_model_preparation_time': 0.006,
 'eval_flat_accuracy': 0.9631481481481482,
 'eval_accuracy': 0.7213333333333334,
 'eval_micro_precision': 0.8433548387096774,
 'eval_micro_recall': 0.8938730853391685,
 'eval_micro_f1': 0.8678794316823795,
 'eval_macro_precision': 0.7501549261163637,
 'eval_macro_recall': 0.8944294497939433,
 'eval_macro_f1': 0.801098295802687,
 'eval_weighted_precision': 0.8512035355191675,
 'eval_weighted_recall': 0.8938730853391685,
 'eval_weighted_f1': 0.8705037184239528,
 'eval_runtime': 129.7731,
 'eval_samples_per_second': 23.117,
 'eval_steps_per_second': 2.89}

In [34]:
trainer.push_to_hub(commit_message="Training complete")

events.out.tfevents.1731310060.d3ae0a06c786.8191.1:   0%|          | 0.00/978 [00:00<?, ?B/s]

events.out.tfevents.1731293946.d3ae0a06c786.2966.0:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.12G [00:00<?, ?B/s]

events.out.tfevents.1731294250.d3ae0a06c786.2966.1:   0%|          | 0.00/4.18k [00:00<?, ?B/s]

events.out.tfevents.1731294420.d3ae0a06c786.8191.0:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

events.out.tfevents.1731310190.d3ae0a06c786.8191.2:   0%|          | 0.00/946 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ebinna/multi_cls_mamba2-780m/commit/c076d3f6f27fc3cba7da30122d057145af83583b', commit_message='Training complete', commit_description='', oid='c076d3f6f27fc3cba7da30122d057145af83583b', pr_url=None, pr_revision=None, pr_num=None)

In [35]:
response = model.predict(text, tokenizer)
print(response)

['Systems']


In [36]:
ebinna = MambaTextClassification.from_pretrained("ebinna/"+output_dir)
ebinna.to("cuda")

tk = AutoTokenizer.from_pretrained("ebinna/"+output_dir)
tk.pad_token_id = tk.eos_token_id

config.json:   0%|          | 0.00/267 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.12G [00:00<?, ?B/s]

Newly initialized embedding: set()


tokenizer_config.json:   0%|          | 0.00/4.84k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

In [37]:
res = ebinna.predict(text, tk)
print(res)

['Systems']


In [38]:
wandb.finish()

eval/accuracy,█▄▁███▄
eval/flat_accuracy,▂▃▁▇██▃
eval/loss,▁▂▆▇▇▇█
eval/macro_f1,▁▇▇▄██▄
eval/macro_precision,█▃▃▂▇▇▁
eval/macro_recall,▁█████▇
eval/micro_f1,▁▅▃▇██▅
eval/micro_precision,█▁▁▄▄▄▂
eval/micro_recall,▁█▇▇██▇
eval/model_preparation_time,▁
eval/runtime,██████▁


In [39]:
print(1)

1
